In [23]:
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt
model = KeyedVectors.load_word2vec_format('50thousand.txt')

In [37]:
for key, value in model.most_similar('FRAM', topn=100000):
    print(key)

KeyError: "Key 'FRAM' not present in vocabulary"

In [ ]:
# 导入所需的库
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

# 定义需要计算相似性的词汇
material = ['BaTiO3','PZT','PVDF','LiNbO3','BiFeO3']
application = ['sensors','actuators','filters','capacitors']
mechanism = ['polarization','strain','capacitive']
structure = ['nanostructure','layered','composite']
property = ['piezoelectricity']
characterization = ['SEM','TEM','XRD','AFM','IR']
preparation = ['CVD','PLD','PVD','MBE']
target_words = ['material','application','mechanism','structure','property'
               ,'characterization','preparation']

# 计算余弦相似性
similarities = {}

# 计算各类别词汇与目标词汇的相似性
categories = {'material':material,'application':application,'mechanism':mechanism,
              'structure':structure,'property':property,
              'characterization':characterization,'preparation':preparation}

for target_word, category in categories.items():
    if target_word in target_words:
        for word in category:
            if word in model and target_word in model:
                similarities[(word, target_word)] = model.similarity(word, target_word)

# 计算target_words之间的相似性
for i in range(len(target_words)):
    for j in range(i + 1, len(target_words)):
        word1 = target_words[i]
        word2 = target_words[j]
        if word1 in model and word2 in model:
            similarities[(word1, word2)] = model.similarity(word1, word2)

# 创建图
G = nx.Graph()

# 添加节点
all_words = material + application + mechanism + structure + property + characterization + preparation + target_words
for word in all_words:
    G.add_node(word)

# 添加边并设置宽度
edges = []
for (word1, word2), sim in similarities.items():
    G.add_edge(word1, word2, weight=sim * 10)  # 乘以10使边的宽度更明显
    edges.append((word1, word2))

# 绘制图
plt.figure(figsize=(18, 18), dpi=1000)
pos = nx.spring_layout(G, seed=42, k=0.5)  # 增加k值以增大节点间距

# 根据权重绘制边
weights = [G[u][v]['weight'] for u, v in edges]

# 绘制边，设置边颜色和透明度
nx.draw_networkx_edges(G, pos, edgelist=edges, width=[weight * 2 for weight in weights], edge_color='bisque', alpha=0.6)  # 乘以2增加宽度

# 绘制特定节点为黄色
yellow_nodes = target_words
other_nodes = set(G.nodes) - set(yellow_nodes)

# 绘制节点和标签
nx.draw_networkx_nodes(G, pos, nodelist=yellow_nodes, node_size=3000, node_color='skyblue')  # 设置节点颜色为黄色
nx.draw_networkx_nodes(G, pos, nodelist=other_nodes, node_size=3000, node_color='lightgray')  # 设置其他节点颜色
nx.draw_networkx_labels(G, pos, font_size=12)

plt.title('Knowledge Graph Based on Cosine Similarity')
plt.axis('off')  # 关闭坐标轴

plt.savefig('ferroelectricity-knowledge-graph.png')
plt.show()
